## Config

In [1]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

# Filepaths
dataPath = 'data/artistsKnown.csv'

## Log In

In [4]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Collect new artists

In [3]:
# Collect names of new artists from Music Festival Wizard
artistsNew = fetchMusicFestivalWizard()

In [6]:
# Collect artists from an existing favorited playlist
playlistToFetch = getPlaylistID(credentials, 'anti pop')
artistsNew = fetchTracksFromPlaylist(credentials, playlistToFetch)
artistsNew

,artistName,artistID,trackName,release_date,songURI
0,Mercury,6aIZIwesnYKR6ZN1fIerPZ,heaven,2023-06-01,spotify:track:5gEoep9RwLEy8hFJMne9ad
1,Portraits Of Tracy,34QizBjxVeMcA5HlZpKxxE,Found!,2023-05-26,spotify:track:1FgY491AT8WYmMgZgWLfkz
2,Kari Faux,4c2ighP1wj8E5dVGJDCOiB,WHITE CAPRICE,2023-05-26,spotify:track:3Q0h7wkHLIlwySLGZYqFPu
3,Gangsta Boo,3ppZNqihWOzuH4A0f4KmeP,WHITE CAPRICE,2023-05-26,spotify:track:3Q0h7wkHLIlwySLGZYqFPu
4,Ray Laurél,7gW3GsnBSoT6q9YQUstlzA,U SAID!,2023-05-19,spotify:track:3S2FcwSlUhob3FAdHgDwWc
...,...,...,...,...,...
119,Sharktank,6BcWGwvuWOW4F9IYBvWR6K,H-A-P-P-Y,2023-03-24,spotify:track:7jseeQmEFs7vp8pIQcXkaO
120,modernlove.,2u17Ej1u0JHyRsstmofsrh,until my heart stops beating,2023-03-24,spotify:track:4lLD8aU0V105kUOYj3h4sJ
121,maxime.,4jd8Wp3Os5tXFV0NYm1570,‎wedgie,2023-04-14,spotify:track:7rG0ambeXy4HZsTCuFnAgX
122,Spencer.,20PdOyVym0ZZrOB1rfcvCR,4evaaa,2023-03-24,spotify:track:0uq8z2z0DUJNRgxv3DfqT3


## Process new artists

In [5]:
# Filter out known artists who we have already listened to
if os.path.exists(dataPath):
    artistsKnown = pd.read_csv(dataPath, index_col=0)
else:
    artistsKnown = pd.DataFrame(columns=['artistName', 'artistID', 'artistPopularity', 'artistFollowers', 'artistGenre'])
artistsNew = artistsNew.loc[~artistsNew.artistName.isin(artistsKnown['artistName'])]
# Search Spotify for new artist info
artistsNew = artistsNew.apply(fetchArtistInfo, axis=1)
# Ignore artists who did not turn up in Spotify
artistsNew = artistsNew.dropna(subset='artistID')
# Drop artists whose name was not an exact match with a known artist but who have the same artistID
artistsNew = artistsNew.loc[~artistsNew.artistID.isin(artistsKnown['artistID'])]

,artistName


## Get top songs

In [83]:
# Get their top three songs in last 5 years
songURIs = fetchTopSongs(artistsNew['artistID'], maxAge=5)
# How many songs are in this round?
print(f"There are {len(songURIs)} songs about to be added")

There are 6 about to be added


## Put songs in playlist

In [114]:
# Warning: This will make a new playlist
playlistID = getPlaylistID('New Artists to Explore')
# Add songs to the playlist
addToPlaylist(songURIs, playlistID)

## Update the list of known artists

In [126]:
# Append artistsNew to artistsKnown.csv 
allArtists = pd.concat([artistsKnown, artistsNew], axis=0, ignore_index=True)
allArtists.to_csv(dataPath)

## Create playlist to drop good songs into

In [118]:
# Cull exploreArtists into ArtistHasPotential playlist
winnersPlaylist = getPlaylistID('New Artists I Liked')
print(f'Now drop songs you like into this playlist: {winnersPlaylist}')

Now drop songs you like into this playlist: 5bZPe8Gkc9jUADiDXXlm4V
